In [70]:
# import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
# do more expensive airbnbs have better ratings than cheaper ones?? 
# expand: which area ?? 

In [71]:
# 2014
may_2014 = pd.read_csv(open('tomslee_airbnb_san_diego_0033_2014-05-25.csv'))
november_2014 = pd.read_csv(open('tomslee_airbnb_san_diego_0063_2014-11-03.csv'))
# 2015
august_2015 = pd.read_csv(open('tomslee_airbnb_san_diego_0142_2015-08-22.csv'))
october1_2015 = pd.read_csv(open('tomslee_airbnb_san_diego_0169_2015-10-03.csv'))
october2_2015 = pd.read_csv(open('tomslee_airbnb_san_diego_0187_2015-10-23.csv'))
december1_2015 = pd.read_csv(open('tomslee_airbnb_san_diego_0222_2015-12-03.csv'))
december2_2015 = pd.read_csv(open('tomslee_airbnb_san_diego_0242_2015-12-16.csv'))
# 2016
january_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0289_2016-01-19.csv'))
february_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0328_2016-02-22.csv'))
march_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0351_2016-03-23.csv'))
april_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0393_2016-04-18.csv'))
may_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0427_2016-05-22.csv'))
june_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0468_2016-06-21.csv'))
july_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0496_2016-07-19.csv'))
august_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0531_2016-08-22.csv'))
september_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0573_2016-09-19.csv'))
october_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0617_2016-10-22.csv'))
november_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0656_2016-11-25.csv'))
december_2016 = pd.read_csv(open('tomslee_airbnb_san_diego_0713_2016-12-25.csv'))
# 2017
january_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_0786_2017-01-15.csv'))
february_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_0865_2017-02-17.csv'))
april_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_1050_2017-04-09.csv'))
may_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_1194_2017-05-06.csv'))
june_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_1316_2017-06-11.csv'))
july_2017 = pd.read_csv(open('tomslee_airbnb_san_diego_1436_2017-07-11.csv'))

# merge all datasets together
#df_2014 = [may_2014, november_2014]
df_2014 = pd.concat([may_2014, november_2014])
#df_2015 = [august_2015, october1_2015, october2_2015, december1_2015, december2_2015]
df_2015 = pd.concat([august_2015, october1_2015, october2_2015, december1_2015, december2_2015])
#df_2016 = [january_2016, february_2016, march_2016, april_2016, may_2016, june_2016, july_2016, august_2016, september_2016, october_2016, november_2016, december_2016]
df_2016 = pd.concat([january_2016, february_2016, march_2016, april_2016, may_2016, june_2016, july_2016, august_2016, september_2016, october_2016, november_2016, december_2016])
#df_2017 = [january_2017, february_2017, april_2017, may_2017, july_2017]
df_2017 = pd.concat([january_2017, february_2017, april_2017, may_2017, july_2017])
frames = [df_2014, df_2015, df_2016, df_2017]
df = pd.concat(frames)

# drop all the unnecessary columns
df.drop(['survey_id', 'bathrooms','minstay','location','country', 'city','borough', 'host_id', 'last_modified', 'latitude', 'longitude', 'room_id'], axis=1, inplace=True)

# drop rows don't have a rating or price
df.dropna(inplace = True, subset = { 'overall_satisfaction', 'price'})

# drop rows that don't have any reviews
#df = df[df['reviews'] != 0]

# sort prices from least expensive to most expensive
df = df.sort_values(['price'], ascending=1)

# reset indices
#df = df.reset_index(drop=True)
print (df)

      accommodates  bedrooms      neighborhood  overall_satisfaction   price  \
3646           2.0       1.0       Mission Bay                   5.0     0.0   
6386           6.0       3.0        North City                   0.0    10.0   
8717           2.0       1.0       Kearny Mesa                   5.0    10.0   
7908           1.0       1.0           Midtown                   0.0    10.0   
15             6.0       3.0        North City                   0.0    10.0   
6390           8.0       3.0     Pacific Beach                   5.0    10.0   
5625           2.0       1.0       North Hills                   4.5    10.0   
349            2.0       1.0           Midtown                   0.0    10.0   
8071           1.0       1.0           Midtown                   5.0    10.0   
6392           4.0       0.0   Gaslamp Quarter                   5.0    10.0   
128            1.0       1.0           Midtown                   5.0    10.0   
7943           1.0       1.0           M

In [72]:
# create dictionary where key is neighborhood and value is number of times it popped up in listing 
# bias could be generated based off of number of reviews and corresponding satisfaction level
# 

In [73]:
# get a range of prices 
#print (something2)
df['price'].value_counts().plot.hist()
a = plt.gcf()